In [57]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import geopy.distance

In [58]:
poe_path = '../Data/PoE/2_POE53_from_jenny.csv'
hosp_path = '../Data/Hospitals/hospital_map.csv'
Poe_df = pd.read_csv(poe_path)
hos_all_df = pd.read_csv(hosp_path)

In [59]:
Poe_df.dropna(inplace=True)
hos_all_df.dropna(inplace=True)
Poe_df.reset_index(drop=True, inplace =True)
hos_all_df.reset_index(drop=True,inplace=True)


In [60]:
hos_df=hos_all_df[hos_all_df['Level'].isin(["Regional Referral Hospital","National Referral Hospital"])].copy()
hos_hc4_df=hos_all_df[hos_all_df['Level'].isin(["Regional Referral Hospital","National Referral Hospital","Hospital",'Health Centre IV'])].copy()

In [61]:
hos_df.rename(columns={'Health Facility':'facility_name'},inplace=True)
hos_hc4_df.rename(columns={'Health Facility':'facility_name'},inplace=True)

In [62]:
len(hos_df)

16

## PoE to hospital


In [63]:
# Create matrix to hold the distances
poe_hospitals_matrix = np.zeros((Poe_df.shape[0],hos_df.shape[0]))
poe_hospitals_matrix.shape

(49, 16)

In [64]:
# Dictionary that maps PoE and facility naes to index (need that as we are going to convert the df to numpy array)
hos_column_dict =  {k: v for v, k in enumerate(hos_df.facility_name)}
poe_column_dict = {k: v for v, k in enumerate(Poe_df.PoEs)}

In [65]:
#fuction to access the index from the above dictionaries
def get_index(poe_name, hos_name):
    return poe_column_dict[poe_name], hos_column_dict[hos_name]

In [66]:
#Function to compute the distance from Poe to hospitals
def compute_PoE_Hospitals_dist():
    for i in Poe_df.PoEs.values:    
        for j in hos_df.facility_name.values:
            coords_1 = (Poe_df[Poe_df['PoEs']==i]['LON'].values[0], Poe_df[Poe_df['PoEs']==i]['LAT'].values[0])
            coords_2 = (hos_df[hos_df['facility_name']==j]['lon'].values[0], hos_df[hos_df['facility_name']==j]['lat'].values[0])


            poe_hospitals_matrix[get_index(i,j)] = geopy.distance.geodesic(coords_1, coords_2).km
    return poe_hospitals_matrix
poe_hospitals_matrix = compute_PoE_Hospitals_dist()

In [67]:
## Get top 3 hospitals closest to each PoE 

hos_column_dict1 =  {v: k for v, k in enumerate(hos_df.facility_name)}
df = pd.DataFrame()
df['index']= range(poe_hospitals_matrix.shape[0])
df['top1'] = np.nan
df['top2'] = np.nan
df['top3'] = np.nan
df['top1_dist'] = np.nan
df['top2_dist'] = np.nan
df['top3_dist'] = np.nan
df['category'] = np.nan

    
for i in range(poe_hospitals_matrix.shape[0]):
    poe_hos = poe_hospitals_matrix[i]

  
    top_3 = np.argsort(poe_hos)[:3]
 
    hos_column_dict1 =  {v: k for v, k in enumerate(hos_df.facility_name)}
    hosp1 = hos_column_dict1[top_3[0]]
    
    hosp2 = hos_column_dict1[top_3[1]]
    hosp3 = hos_column_dict1[top_3[2]]


    mask = df['index'] == i
    df['top1'][mask] = hosp1
    df['top2'][mask] = hosp2
    df['top3'][mask]  = hosp3
    
    df['top1_dist'][mask] = poe_hos[top_3[0]]
    df['top2_dist'][mask] = poe_hos[top_3[1]]
    df['top3_dist'][mask] = poe_hos[top_3[2]]
    
    
    df['category'] = "PoE"
    df['level'] = "PoE"
  
df['Long'] = Poe_df['LON']
df['lat']  = Poe_df['LAT']
df['hospital or poe']= Poe_df['PoEs']    

## Hospital  to PoE


Find distance from Hosptitals to PoE

In [68]:
hos_hc4_df

,Unnamed: 0,facility_name,Level,Ownership,lat,lon
1,1,Buikwe St. Charles Lwanga HOSPITAL,Hospital,Private Not For Profit,0.339440,33.031810
6,6,Kawolo HOSPITAL,Hospital,Government,0.367830,32.945080
11,11,Makonge HC III,Health Centre IV,Private Not For Profit,0.270890,33.149990
22,24,Nkokonjeru HOSPITAL,Hospital,Private Not For Profit,0.235990,32.923020
25,27,St. Francis Nyenga HOSPITAL,Hospital,Private Not For Profit,0.395050,33.140650
...,...,...,...,...,...,...
5933,6135,Senta Medicare HC IV,Health Centre IV,PFP,0.234496,32.600494
5943,6145,Spring Medicare HC IV,Health Centre IV,PFP,0.409001,32.485988
6007,6209,URO Care Hospital,Hospital,PFP,0.368888,32.534516
6011,6213,Victoria Medical Services HC IV,Health Centre IV,PFP,0.214877,32.577385


In [69]:
hos_hc4_column_dict =  {k: v for v, k in enumerate(hos_hc4_df.facility_name)}
poe_column_dict = {k: v for v, k in enumerate(Poe_df.PoEs)}

In [70]:
#Matrix to hold the distances
hospitals_poe_matrix = np.zeros((hos_hc4_df.shape[0],Poe_df.shape[0]))

#Get index of a given PoE and hospital
def get_index2(poe_name, hos_name):
    return hos_hc4_column_dict[hos_name], poe_column_dict[poe_name]


In [71]:
def compute_Hospitals_PoE_dist():
    for i in hos_hc4_df.facility_name.values:    
        for j in Poe_df.PoEs.values:
            coords_1 = (Poe_df[Poe_df['PoEs']==j]['LON'].values[0], Poe_df[Poe_df['PoEs']==j]['LAT'].values[0])
            coords_2 = (hos_hc4_df[hos_hc4_df['facility_name']==i]['lon'].values[0], hos_hc4_df[hos_hc4_df['facility_name']==i]['lat'].values[0])

            hospitals_poe_matrix[get_index2(j,i)] = geopy.distance.geodesic(coords_1, coords_2).km
    return hospitals_poe_matrix

In [72]:
hospitals_poe_matrix = compute_Hospitals_PoE_dist()
hospitals_poe_matrix.shape

(379, 49)

In [73]:
## Get top 3 PoE closest to each Hospitals 

poe_column_dict1 = {v: k for v, k in enumerate(Poe_df.PoEs)}
df1 = pd.DataFrame()
df1['index']= range(hospitals_poe_matrix.shape[0])
df1['top1'] = np.nan
df1['top2'] = np.nan
df1['top3'] = np.nan
df1['top1_dist'] = np.nan
df1['top2_dist'] = np.nan
df1['top3_dist'] = np.nan
df1['category'] = np.nan

    
for i in range(hospitals_poe_matrix.shape[0]):
    poe_hos = hospitals_poe_matrix[i]
  
    
    top_3 = np.argsort(poe_hos)[:3]
    hos_column_dict1 =  {v: k for v, k in enumerate(Poe_df.PoEs)}
    
    hosp1 = hos_column_dict1[top_3[0]]
    hosp2 = hos_column_dict1[top_3[1]]
    hosp3 = hos_column_dict1[top_3[2]]


    mask = df1['index'] == i
    df1['top1'][mask] = hosp1
    df1['top2'][mask] = hosp2
    df1['top3'][mask] = hosp3
    df1['top1_dist'][mask] = poe_hos[top_3[0]]
    df1['top2_dist'][mask] = poe_hos[top_3[1]]
    df1['top3_dist'][mask] = poe_hos[top_3[2]]
    
    
    df1['category'] = "hospital"
    df1['level'] = list(hos_hc4_df['Level'])

In [90]:
top_3

array([1, 0, 3], dtype=int64)

In [74]:
df1['Long'] = list(hos_hc4_df['lon'])
df1['lat'] = list(hos_hc4_df['lat'])
df1['hospital or poe']= list(hos_hc4_df['facility_name'])

## Stack the two dataframes

In [79]:
all_dfs = [df, df1]

main_df = pd.concat(all_dfs).reset_index(drop=True)
main_df.drop('index', axis=1, inplace = True)

In [80]:
main_df.head()

,top1,top2,top3,top1_dist,top2_dist,top3_dist,category,level,Long,lat,hospital or poe
0,Butabika NATIONAL REFERAL HOSPITAL,Naguru Hospital - China Uganda Friendship,Mulago National Hospital- Old NATIONAL REFERAL...,2.356913,6.406779,9.747405,PoE,PoE,32.654000,0.290000,Port Bell
1,Mulago National Hospital- Old NATIONAL REFERAL...,Naguru Hospital - China Uganda Friendship,Butabika NATIONAL REFERAL HOSPITAL,31.267736,32.280677,34.678985,PoE,PoE,32.443055,0.044721,Entebbe Airport
2,Jinja Regional Ref HOSPITAL,Butabika NATIONAL REFERAL HOSPITAL,Naguru Hospital - China Uganda Friendship,15.598351,54.424288,59.995283,PoE,PoE,33.145901,0.279041,Kiyindi
3,Butabika NATIONAL REFERAL HOSPITAL,Naguru Hospital - China Uganda Friendship,Mulago National Hospital- Old NATIONAL REFERAL...,22.941662,27.927281,31.135860,PoE,PoE,32.803276,0.143008,Katosi
4,Masaka REGIONAL REF HOSPITAL,Mbarara REGIONAL REF HOSPITAL,Mubende RR HOSPITAL,72.753453,91.675881,149.082476,PoE,PoE,31.416706,-1.000000,Mutukula


In [81]:
main_df = main_df[['hospital or poe','category', 'level','Long', 'lat', 'top1', 'top2','top3','top1_dist','top2_dist','top3_dist']]

In [83]:
main_df.to_csv('../Data/Results/PoE-RRH_facilities_top3.csv')

In [48]:
main_df.head()

,hospital or poe,category,Long,lat,top1,top2,top3,top1_dist,top2_dist,top3_dist
0,Port Bell,PoE,32.654000,0.290000,Naguru Hospital - China Uganda Friendship,Jinja Regional Ref HOSPITAL,Masaka REGIONAL REF HOSPITAL,6.406779,62.373754,117.515295
1,Entebbe Airport,PoE,32.443055,0.044721,Naguru Hospital - China Uganda Friendship,Masaka REGIONAL REF HOSPITAL,Jinja Regional Ref HOSPITAL,32.280677,86.153220,91.794509
2,Kiyindi,PoE,33.145901,0.279041,Jinja Regional Ref HOSPITAL,Naguru Hospital - China Uganda Friendship,Mbale REGIONAL REF HOSPITAL,15.598351,59.995283,136.087597
3,Katosi,PoE,32.803276,0.143008,Naguru Hospital - China Uganda Friendship,Jinja Regional Ref HOSPITAL,Masaka REGIONAL REF HOSPITAL,27.927281,51.944846,126.609952
4,Mutukula,PoE,31.416706,-1.000000,Masaka REGIONAL REF HOSPITAL,Mbarara REGIONAL REF HOSPITAL,Mubende RR HOSPITAL,72.753453,91.675881,149.082476
